# RSA Encryption
By: Jonathan Oliveros

The usefulness of the Rivest-Shamir-Adleman (RSA) encryption system is tremendous, since all of what is being sent back and forth through the internet depends on it. The problem is as follows: if someone wants to send a message publicly, how can we make sure that only the one that the message is intended for can see it? The RSA is (as far as we know) creates a hard to invert one way function that can only be solved if the recipient has the key. This notebook was created to show the steps on how it works.

I will be using the steps for encrypting and decrypting the message from this site:

https://courses.cs.vt.edu/~cs5204/fall00/protection/rsa.html

Before getting started, lets import the required packages that will be used.

In [1]:
from IPython.display import Markdown as md
import math

## 1. Represent the message as an integer between 0 and (n-1). Large messages can be broken up into a number of blocks. Each block would then be represented by an integer in the same range. 

I decided to use the conversion to ASCII for simplicity's sake. chr() converts an int to its ASCII representation and ord() converts the ASCII character to an integer.

In [2]:
alphanum = {}

for i in range(65,91):
    alphanum[chr(i)] = i
print(alphanum)

{'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72, 'I': 73, 'J': 74, 'K': 75, 'L': 76, 'M': 77, 'N': 78, 'O': 79, 'P': 80, 'Q': 81, 'R': 82, 'S': 83, 'T': 84, 'U': 85, 'V': 86, 'W': 87, 'X': 88, 'Y': 89, 'Z': 90}


In [3]:
code = 'EUCLID'
code = code.upper()
cipher = []
#loop in pairs of 2 from 0 to length of word. if word has odd length, then append last letter.
for i in range(1,len(code),2):
    first = ord(code[i - 1])
    second = ord(code[i])
    cipher.append(str(first) + str(second))
if len(code) % 2 != 0:
    cipher.append(str(ord(code[len(code) - 1])))
print(cipher)

['6985', '6776', '7368']


Now that the word has been converted, lets clear it out to show that it is not printing out the result from the input 

In [4]:
code = ''
print(code)

## Useful Methods

The following two methods are part of what's called the successive algorithm. This will be useful in computing really large numbers mod n. A comparison chart on its efficiency can be found here https://github.com/JonathanOliveros/successive-squares

In [5]:
def convert_db(decimal):
        """
        Converts a decimal number to a binary number.
        """
        lst = []
        num = decimal
        while num > 0:
            lst.append(num % 2) # add remainder to binary digit
            num = math.floor(num / 2) # divide by 2 for next iteration
        lst.reverse() #reverse list to put binary number in correct order
        return lst

def get_mod(base, exp, m):
        """
        Uses the successive squares algorithm to reduce very large integers
        mod m.

        returns:

        reduced number mod m
        """
        
        binary_num = convert_db(exp)
        num_lst = [] # create a list that holds (base**(2**i)) mod m 
        result = 1
        num_lst.append(base)# add base number to list since 2**0 == 1
        for i in range(1,len(binary_num)): # start i at 1 to get rest of the list.
            num_lst.append((num_lst[i - 1]**2 % m)) # square the result of the previous number on list and mod m
        num_lst.reverse() # reverse list to match up with binary number list order
        for j in range(0, len(binary_num)):
            if binary_num[j] == 1: #the 1's and 0's are flags to see if the index in num_lst can be multiplied to result 
                result *= num_lst[j]
        return result % m # the result is easier to reduce mod m and is congruent to original number


In [6]:
def gcd(a, b):
    while a != 0:
        q, r = b//a, b%a
        b, a = a, r
    gcd = b
    return gcd

In [7]:
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

The extended Euclidian Algorithm comes from:

https://brilliant.org/wiki/extended-euclidean-algorithm/

This algorithm takes the gcd and solves the equation $ax + by = gcd(a,b)$, which is called Bezout's Identity. More information can be found here:

https://en.wikipedia.org/wiki/B%C3%A9zout%27s_identity

the gcd method above is only uses the euclidian algorithm part of the egcd method.

## How to Determine Appropriate Values for e, d, and n

1. Choose two very large (100+ digit) prime numbers. Denote these numbers as p and q.
Set n equal to p * q.
2. Choose any large integer, d, such that GCD($d, ((p-1) * (q-1))) = 1$ (For readability's sake let $\phi = (p-1)*(q-1)$)
3. Find e such that e * d $\equiv$ 1 (mod ((p-1) * (q-1))) 

In [8]:
#choose two primes
p = 7097693

q = 7098937
# choose d
d = 7154999

(Notice that d is a prime as well. This was chosen intentionally because for any integer a, gcd(a,d) will always be 1 due to the nature of primes itself.)

In [9]:
n = p * q
phi = (p-1) * (q-1)

#check that GCD(phi, d) = 1
print(gcd(d, phi))

1


In [10]:
Bezout = egcd(phi,d)
print(Bezout)

(1, 658753, -4638990027865)


In [11]:
md(f"""By Bezout's Identity, this means that {Bezout[1]} $\cdot$ {phi} + {Bezout[2]} $\cdot$ {d} = {Bezout[0]}""")

By Bezout's Identity, this means that 658753 $\cdot$ 50386061255712 + -4638990027865 $\cdot$ 7154999 = 1

In [12]:
md(f"""Therefore e = {Bezout[2]} and is the inverse coefficient that solves
   $$
    {Bezout[2]} * {d} \equiv {Bezout[0]} \ mod \ {phi}
   $$""")

Therefore e = -4638990027865 and is the inverse coefficient that solves
   $$
    -4638990027865 * 7154999 \equiv 1 \ mod \ 50386061255712
   $$

## 2. Encrypt the message by raising it to the $e^{th}$ power modulo n. The result is a ciphertext message C. 

In [13]:
#mod e by phi to make sure number is positive
e = Bezout[2] % phi
print(e)

45747071227847


In [14]:
C = []
for i in cipher:
    C.append(get_mod(int(i), e, n))
print(C)

[25636130755670, 31991081607741, 45749641709595]


## 3. To decrypt ciphertext message C, raise it to another power d modulo n

In [15]:
decrypted = []
for i in C:
    decrypted.append(str(get_mod(i, d, n)))
print(decrypted)

['6985', '6776', '7368']


Comparing to the original cipher:

In [16]:
print(cipher)

['6985', '6776', '7368']


From this we can see that they are equivalent and the by decrypting the cipher we get the original encoded word.

In [17]:
split = 2
result = ""
for i in decrypted:
    if len(i) == 2:
        result += chr(int(i))
    else:
        a, b = int(i[:split]), int(i[split:])
        result += chr(a) + chr(b)
print(result)

EUCLID
